In [1]:
import torch
import torchvision
import torchvision.transforms as transforms

In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [3]:
training_data = datasets.FashionMNIST(
	root="data",
	train=True,
	download=True,
 	transform=ToTensor()	
)

testing_data = datasets.FashionMNIST(
	root="data",
	train=False,
	download=True,
	transform=ToTensor()
)

In [4]:
batch_size = 64
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(testing_data, batch_size=batch_size)

In [5]:

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [6]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cpu device


In [9]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
			nn.Linear(28*28, 512),
			nn.ReLU(),
   			nn.Linear(512, 512),
	   		nn.ReLU(),
   			nn.Linear(512, 10),
		)
        
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
    
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [11]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 1e-2)


In [12]:
def train(dataloader, model, optimiser, loss_fn):
    size = len(dataloader.dataset)
    model.train()
    for batch , (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        
        pred = model(X)
        loss = loss_fn(pred, y)
        
        loss.backward()
        optimiser.step()
        optimiser.zero_grad()
        
        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")
        

In [13]:
def test(dataloader, model, loss_fn):
	size = len(dataloader.dataset)
	num_batches = len(dataloader)
	model.eval()
	test_loss, correct = 0, 0
	with torch.no_grad():
		for X, y in dataloader:
			X, y = X.to(device), y.to(device)
			pred = model(X)
			test_loss += loss_fn(pred, y).item()
			correct += (pred.argmax(1) == y).type(torch.float).sum().item()

	test_loss /=num_batches
	correct /= size
	print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [14]:
epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, optimizer, loss_fn)
    test(test_dataloader, model, loss_fn)

Epoch 1
-------------------------------
loss: 2.308882 [    0/60000]
loss: 2.151703 [ 6400/60000]
loss: 1.748303 [12800/60000]
loss: 1.454899 [19200/60000]
loss: 1.107123 [25600/60000]
loss: 1.018307 [32000/60000]
loss: 0.994534 [38400/60000]
loss: 0.871709 [44800/60000]
loss: 0.859045 [51200/60000]
loss: 0.809340 [57600/60000]
Test Error: 
 Accuracy: 71.2%, Avg loss: 0.789373 

Epoch 2
-------------------------------
loss: 0.782626 [    0/60000]
loss: 0.850298 [ 6400/60000]
loss: 0.581622 [12800/60000]
loss: 0.784934 [19200/60000]
loss: 0.660822 [25600/60000]
loss: 0.637098 [32000/60000]
loss: 0.710372 [38400/60000]
loss: 0.681949 [44800/60000]
loss: 0.693674 [51200/60000]
loss: 0.641514 [57600/60000]
Test Error: 
 Accuracy: 77.9%, Avg loss: 0.631220 

Epoch 3
-------------------------------
loss: 0.555185 [    0/60000]
loss: 0.657388 [ 6400/60000]
loss: 0.434221 [12800/60000]
loss: 0.670135 [19200/60000]
loss: 0.581655 [25600/60000]
loss: 0.559000 [32000/60000]
loss: 0.591813 [38400/

In [15]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


In [16]:
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth", weights_only=True))

<All keys matched successfully>

In [18]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = testing_data[100][0], testing_data[100][1]
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Dress", Actual: "Dress"
